# 파일 불러오기

In [1]:
import glob
import os
import re
import tensorflow as tf

# 다운받은 파일들을 경로를 지정
txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'

#txt_fila_path에 있는 파일들을 리스트 형식으로 저장합니다
#glob.glob는 무슨 기능일까요? 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다. 사용할 파일들이 많을 때 유용하다.
txt_list = glob.glob(txt_file_path)

print(txt_list)

['/aiffel/aiffel/lyricist/data/lyrics/janisjoplin.txt', '/aiffel/aiffel/lyricist/data/lyrics/nursery_rhymes.txt', '/aiffel/aiffel/lyricist/data/lyrics/dickinson.txt', '/aiffel/aiffel/lyricist/data/lyrics/nicki-minaj.txt', '/aiffel/aiffel/lyricist/data/lyrics/disney.txt', '/aiffel/aiffel/lyricist/data/lyrics/bob-marley.txt', '/aiffel/aiffel/lyricist/data/lyrics/beatles.txt', '/aiffel/aiffel/lyricist/data/lyrics/dj-khaled.txt', '/aiffel/aiffel/lyricist/data/lyrics/kanye.txt', '/aiffel/aiffel/lyricist/data/lyrics/patti-smith.txt', '/aiffel/aiffel/lyricist/data/lyrics/drake.txt', '/aiffel/aiffel/lyricist/data/lyrics/r-kelly.txt', '/aiffel/aiffel/lyricist/data/lyrics/rihanna.txt', '/aiffel/aiffel/lyricist/data/lyrics/cake.txt', '/aiffel/aiffel/lyricist/data/lyrics/bruno-mars.txt', '/aiffel/aiffel/lyricist/data/lyrics/leonard-cohen.txt', '/aiffel/aiffel/lyricist/data/lyrics/ludacris.txt', '/aiffel/aiffel/lyricist/data/lyrics/kanye-west.txt', '/aiffel/aiffel/lyricist/data/lyrics/eminem.txt', 

# 파일에 있는 말뭉치 꺼내기

In [2]:
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, 'r') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기 : ", len(raw_corpus))
print("샘플 : \n", raw_corpus[:3])

데이터 크기 :  187088
샘플 : 
 ["Busted flat in Baton Rouge, waitin' for a train", "And I's feelin' near as faded as my jeans", 'Bobby thumbed a diesel down, just before it rained']


# 데이터 정제

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'([!.?,¿])', r' \1 ', sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"([^a-zA-Z!.?,¿])+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence +' <end>'
    return sentence

처음에 for문에서 토큰 15개 이상 되는 시퀀스를 제거하려고 

if preprocessed_sentence > 15: continue

이렇게 코드를 넣었다. 
그랬더니 모든 데이터가 제거되어서 시작 토큰과 끝 토큰만 남게 되었다. 곧 문자 하나하나를 일일이 길이로 카운터 하기 때문이라는 것을 알게 되었다. tf.keras.preprocessing.text.Tokenizer로 말뭉치를 텐서로 변환할 때 15개 토큰을 초과하는 것을 제거하는 것으로 해봐야 되겠다. 단 패딩 작업 이전에는 수행을 해야 한다.

In [4]:
corpus = []

for sentence in raw_corpus:
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
print(corpus[:3])

['<start> busted flat in baton rouge , waitin for a train <end>', '<start> and i s feelin near as faded as my jeans <end>', '<start> bobby thumbed a diesel down , just before it rained <end>']


In [5]:
def Tokenizer(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=12000, oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    print(tensor[0])
    tensor_15 = []
    for tensor_line in tensor:
        if len(tensor_line) <= 15:
            tensor_15.append(tensor_line)
        else:
            continue
    tensor_15 = tf.keras.preprocessing.sequence.pad_sequences(tensor_15, padding='post', maxlen=20)
    print(tensor_15, tokenizer)
    return tensor_15, tokenizer

tensor_15, tokenizer = Tokenizer(corpus)

[3, 3603, 1686, 13, 7052, 3989, 1094, 26, 8, 675, 2]
[[   3 3603 1686 ...    0    0    0]
 [   3    7    4 ...    0    0    0]
 [   3  798 7658 ...    0    0    0]
 ...
 [   3    4   20 ...    0    0    0]
 [   3    4   20 ...    0    0    0]
 [   3    4   20 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f652d5a5b50>


In [6]:
tensor_15[:3]

array([[   3, 3603, 1686,   13, 7052, 3989, 1094,   26,    8,  675,    2,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   3,    7,    4,   15,  507,  842,   77, 2583,   77,   12,  942,
           2,    0,    0,    0,    0,    0,    0,    0,    0],
       [   3,  798, 7658,    8, 6042,   56,   34,  181,   10, 4569,    2,
           0,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)

In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx > 10: break

1 : <unk>
2 : end
3 : start
4 : i
5 : the
6 : you
7 : and
8 : a
9 : to
10 : it
11 : me


# 평가 데이터셋 분리

In [8]:
src_input = tensor_15[:, :-1]
tga_input = tensor_15[:, 1:]

print(src_input[0])
print(tga_input[0])

[   3 3603 1686   13 7052 3989 1094   26    8  675    2    0    0    0
    0    0    0    0    0]
[3603 1686   13 7052 3989 1094   26    8  675    2    0    0    0    0
    0    0    0    0    0]


오랜만에 train_test_split를 사용하다 보니까 전반적으로 까먹어서 구글링을 통해서 라이브러리를 불러오는 법을 기억했다.
일단 데이터셋을 만들기 전에 훈련데이터와 테스트 데이터로 나누는 것부터 하기록 했다.

In [9]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tga_input, test_size=0.2)

# 데이터셋 객체 생성

In [10]:
# 훈련 데에터를 데이터셋 객체로 생성
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = BUFFER_SIZE // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words+1

data_train = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
data_train = data_train.shuffle(BUFFER_SIZE)
data_train = data_train.batch(BATCH_SIZE, drop_remainder=True)
data_train

<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

In [11]:
#256 = 시퀀스 데이터 수량, 문장
#19 = 시퀀스 길이, 단어
# 

In [12]:
dec_train[:2]

array([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)

In [13]:
# 테스트 데이터를 데이터셋 객체로 생성
BUFFER_SIZE = len(enc_val)
BATCH_SIZE = 256
steps_per_epoch = BUFFER_SIZE // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words+1

data_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
data_val = data_val.shuffle(BUFFER_SIZE)
data_val = data_val.batch(BATCH_SIZE, drop_remainder=True)
data_val

<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

# 인공지능 학습

In [14]:
# 모델 구조
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [15]:
model = TextGenerator(tokenizer.num_words+1, 256, 1024)

In [16]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

엄청 힘들었어요. 처음에는 훈련데이터셋 따로 훈련하고 검증데이터셋 따로 훈련하는건가? 하고 생각했어요.
그러다가 전이학습이 생각이 나서 훈련데이터셋으로 훈련한 다음 추가로 검증데이터셋을 훈련하는 건가하고 생각도 했죠.
그런데 아무리 생각해도 EXPLORATION 6번 노드를 진행하기에는 전이학습은 진도가 빠른 감이 있었어요.

결국 고민 끝에 웅제님께 물어봤고, 웅제님이 노드에서 링크(https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)를 타고 들어가라고 했어요. 그랬더니 fit 함수에 validation_data=라는 메소드가 있는 거에요. 혹시나 돌렸는데, val_loss도 함께 나와서 너무 좋았어요.

In [17]:
model.fit(data_train, epochs=10, validation_data=data_val)

Epoch 1/10
536/536 [==============================] - 138s 247ms/step - loss: 2.4232 - val_loss: 2.1522
Epoch 2/10
536/536 [==============================] - 135s 252ms/step - loss: 2.0799 - val_loss: 2.0514
Epoch 3/10
536/536 [==============================] - 136s 253ms/step - loss: 1.9791 - val_loss: 1.9706
Epoch 4/10
536/536 [==============================] - 136s 253ms/step - loss: 1.8937 - val_loss: 1.9160
Epoch 5/10
536/536 [==============================] - 136s 253ms/step - loss: 1.8206 - val_loss: 1.8696
Epoch 6/10
536/536 [==============================] - 136s 253ms/step - loss: 1.7552 - val_loss: 1.8326
Epoch 7/10
536/536 [==============================] - 136s 254ms/step - loss: 1.6937 - val_loss: 1.8021
Epoch 8/10
536/536 [==============================] - 136s 253ms/step - loss: 1.6364 - val_loss: 1.7772
Epoch 9/10
536/536 [==============================] - 136s 253ms/step - loss: 1.5830 - val_loss: 1.7559
Epoch 10/10
536/536 [==============================] - 136s 254m

# 문장 생성

In [18]:
# 문장 생성 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["end"]
    
    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token:break
        if test_tensor.shape[1] >= max_len: break
            
    generated = ''
    
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated

In [19]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'start i love you end '

마지막에 키에러가 났다. index_word를 통해서 보니까 <end>가 end로 되어 있었다.
    
그래서 end_token에 end로 고쳤다.
    
이 프로젝트를 잘 수행하기 위해서 총 3일 정도 공부했다. 금,일,월 그리고 화요일 아침에 조금..
    
14번 프로젝트를 잘 할 수 있을지 걱정이 된다. 일단 부딪쳐 봐야되겠지.
    
아마 프로젝트가 변할 수도 있지만 입력 문장 끝에 문장 생성하는 것이 아닌 입력 문장 사이에 문장을 생성하는 것이 있으면 좋겠다고 생각한다.
    
그러면 내가 쓴 문장에 논리가 안 맞는 것을 찾아서 고쳐써주지 않을까 하는 생각이 들었다.